<a href="https://colab.research.google.com/github/HwangHanJae/Recommendation_Study/blob/main/book_Recommender_Systems/CF_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [173]:
import pandas as pd
import numpy as np

# 사용자 기반 협업 필터링 모델

In [226]:
class UserBaseCF():
  def __init__(self, table):
    """
    R : 원본테이블
    S : 원본테이블에서 평균값을 뺀 테이블
    Peason : 사용자-사용자 유사도
    peergroup : 피어그룹(상관계수가 0보다 크고 자기자신이 아님)
    """
    self.R = table
    self.S = self._get_S()
    self.Pearson = self.R.T.corr(method='pearson')
    self.peergroup = self._find_peer_group()
  def _get_S(self):
    """
    S 테이블을 만드는 함수
    
    """
    Mu = self.R.mean(axis=1)
    return (R.T - Mu).T
  def _find_peer_group(self):
    """
    피어그룹을 찾는 함수
    """
    pearson = self.Pearson
    return pearson[(pearson>0) & (pearson < 1)]

  def _find_item(self,user_id):
    """
    유저가 아직 평가하지 않은 아이템을 찾는 함수
    """
    index = self.R.loc[user_id, self.R.loc[user_id].isna()].index
    return index
  def recommend(self, user_id):
    """
    사용자가 아직 평가하지 않은 아이템을 찾고
    해당 아이템에 대하여 평점을 예측한 뒤 평점이 높은 순으로 인덱스를 반환하는 함수

    """
    #유저가 아직 평가하지 않은 아이템을 찾음
    item_index  = self._find_item(user_id)
    #유저의 평균
    Mu = self.R.mean(axis=1).loc[user_id]
    #피어그룹
    peergroup = self.peergroup.loc[user_id].dropna().index
    #계산
    a  = self.Pearson.loc[user_id, peergroup].dot(self.S.loc[peergroup, item_index])
    b = np.sum(self.Pearson.loc[user_id, peergroup])
    result = (a / b) + Mu
    
    result.sort_values(ascending=False)
    return  result

In [227]:
#예제 테이블(R) 정의

values = [[7,6,7,4,5,4],
         [6,7,None,4,3,4],
         [None, 3,3,1,1,None],
         [1,2,2,3,3,4],
         [1,None,1,2,3,3]]

R = pd.DataFrame(values, columns = [1,2,3,4,5,6], index = [1,2,3,4,5])

R

,1,2,3,4,5,6
1,7.0,6.0,7.0,4,5,4.0
2,6.0,7.0,NaN,4,3,4.0
3,NaN,3.0,3.0,1,1,NaN
4,1.0,2.0,2.0,3,3,4.0
5,1.0,NaN,1.0,2,3,3.0


In [228]:
temp = UserBaseCF(R)

temp.recommend(3)

1    3.343864
6    0.864318
Name: 3, dtype: float64

3번 사용자가 아직 평가하지 않은 아이템은 1, 6번 아이템입니다.

1, 6번 아이템에 대하여 모델은 다음과 같이 예측하였습니다.

- 1번 아이템 : 3.34
- 6번 아이템 : 0.86



# 아이템 기반 협업 필터링 모델

In [298]:

from itertools import combinations_with_replacement

class ItemBaseCF():
  def __init__(self, table):
    self.R  = table
    self.S  = self._get_S()
    self.AdjustedCosine = self._adjusted_cosine()
    self.peergroup = self._find_peer_group()
  def _get_S(self):
    """
    S 테이블을 만드는 함수
    """
    Mu = self.R.mean(axis=1)
    return (R.T - Mu).T
  def _cal_U(self):
    return self.R.notnull()

  def _cal_adujusted_cosine(self, i, j):
    #행렬 S에서 아이템 i, j의 열을 찾음
    I_i = self.S.loc[:, i]
    I_j = self.S.loc[:, j]
    # 두 아이템 i, j에 모두 평가한 유저의 인덱스를 찾음
    u = set((I_i + I_j).loc[(I_i + I_j).notnull()].index)
    
    #평균 중심 행렬(S)에서 sui, suj를 찾음
    #조정된 코사인 유사도 계산
    sui = self.S.loc[u, i]
    suj = self.S.loc[u, j]
    sum_sui_2 = np.sum(np.power(sui, 2))
    sum_suj_2 = np.sum(np.power(suj, 2))
    sqrt_sum_sui = np.sqrt(sum_sui_2)
    sqrt_sum_suj = np.sqrt(sum_suj_2)

    return np.sum((sui * suj)) / (sqrt_sum_sui * sqrt_sum_suj)
  def _adjusted_cosine(self):
    temp = pd.DataFrame()
    for i, j in combinations_with_replacement(self.R.columns, 2):
        temp.loc[i,j] = self._cal_adujusted_cosine(i,j)
        temp.loc[j,i] = temp.loc[i, j]
    return temp
  def _find_peer_group(self):
    """
    피어그룹을 찾는 함수
    """
    adjustedcosine = self.AdjustedCosine
    return adjustedcosine[(adjustedcosine > 0) & (adjustedcosine < 1)]
  def _find_item(self,user_id):
    """
    유저가 아직 평가하지 않은 아이템을 찾는 함수
    """
    index = self.R.loc[user_id, self.R.loc[user_id].isna()].index
    return index
  
  def recommend(self, user_id):
    """
    사용자가 아직 평가하지 않은 아이템을 찾고
    해당 아이템에 대하여 평점을 예측한 뒤 평점이 높은 순으로 인덱스를 반환하는 함수

    """
    #유저가 아직 평가하지 않은 아이템을 찾음
    item_index  = self._find_item(user_id)
    #피어그룹
    result = pd.Series()
    
    for t in item_index:
      peergroup = self.peergroup.loc[t].dropna().index
      a = self.AdjustedCosine.loc[peergroup, t].dot(self.R.loc[user_id, peergroup])
      b = np.sum(self.AdjustedCosine.loc[peergroup, t])
      result.loc[t] = (a/b)
      
    result.sort_values(ascending=False)
    return result  

In [299]:
temp = ItemBaseCF(R)
temp.recommend(3)

<ipython-input-298-484d2aa828cb>:63: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  result = pd.Series()


1    3.0
6    1.0
dtype: float64

3번 사용자가 아직 평가하지 않은 아이템은 1, 6번 아이템입니다.

1, 6번 아이템에 대하여 모델은 다음과 같이 예측하였습니다.

- 1번 아이템 : 3.0
- 6번 아이템 : 1.0



사용자 기반 협업 필터링과 차이점으로는 예측값의 범위가 제한된 범위를 넘어가지 않습니다.

6번 아이템의 예측값을 보면

- 사용자 기반 : 0.86
- 아이템 기반 : 1.0

으로 사용자 기반은 평점의 범위를 넘어가는 것을 볼 수 있습니다.

또한 일반적으로 아이템 기반의 정확도가 사용자 기반보다 높습니다.